<a href="https://colab.research.google.com/github/mohammad2682/FVS-SVM-RKHS/blob/main/SVM_RKHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import svm

In [2]:
!git clone https://github.com/mohammad2682/FVS-SVM-RKHS

Cloning into 'FVS-SVM-RKHS'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 59 (delta 22), reused 41 (delta 17), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [57]:
def sigma(distance):
  return 5 * distance

In [58]:
def K(FV1, FV2):
  dist = np.linalg.norm(FV1.values-FV2.values)
  sig = sigma(dist)
  return np.exp(-1*np.square(dist)/(2*sig))

In [6]:
df = pd.read_csv("FVS-SVM-RKHS/datasets/pima.csv")
df.head()

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,14,175,62,30,0,33.6,0.212,38,tested_positive
1,4,146,78,0,0,38.5,0.520,67,tested_positive
2,15,136,70,32,110,37.1,0.153,43,tested_positive
3,3,107,62,13,48,22.9,0.678,23,tested_positive
4,3,169,74,19,125,29.9,0.268,31,tested_positive


In [45]:
df_test1 = df.iloc[:3, :]
df_test2 = df.iloc[-7:, :]
df_test = pd.concat([df_test1, df_test2]).reset_index(drop=True)
df_test = df_test[["preg", "plas", "class"]]

In [46]:
df_test

,preg,plas,class
0,14,175,tested_positive
1,4,146,tested_positive
2,15,136,tested_positive
3,6,114,tested_negative
4,6,92,tested_negative
5,5,117,tested_negative
6,4,83,tested_negative
7,7,119,tested_negative
8,1,95,tested_negative
9,1,181,tested_positive


In [59]:
FV1 = df.iloc[[0], :-1]
FV2 = df.iloc[[1], :-1]
FV3 = df.iloc[[2], :-1]
k1 = K(FV1, FV3)
k2 = K(FV2, FV2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [60]:
k1

8.159430063067645e-06

In [41]:
LF = abs(1-(np.transpose(k1)*k1*k1)/k2)

IndexError: ignored

In [ ]:
np.var(k)

0.18034651899606274

In [ ]:
k/k

array([[1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
k.T

array([1.02369992e-04, 2.78966560e-34, 6.11001157e-11, 1.23467889e-36,
       1.00000000e+00, 1.10164700e-01, 9.91322789e-01, 2.78966560e-34])

In [ ]:
df['class'].unique()

array(['tested_positive', 'tested_negative'], dtype=object)

In [ ]:
df.groupby("class").agg("count")

,preg,plas,pres,skin,insu,mass,pedi,age
class,,,,,,,,
tested_negative,500,500,500,500,500,500,500,500
tested_positive,268,268,268,268,268,268,268,268
